In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
results_path='../../../griffin_paper/delfi_cancer_detection/Ulz_analysis/scratch_copy/'
metadata_path = '../../../griffin_paper/data/delfi_metadata/analysis/matched_metadata_corrected.txt'
tfx_path = '../../metadata/delfi/analysis/Delfi_tumor_fraction_data.txt'

output_path = 'merged_data/Ulz_cancer_detection_results_merged.txt'

In [3]:
results=pd.DataFrame()
for item in os.listdir(results_path): #import the results
    if item.startswith('.'):
        continue
    result_file = results_path+item
    current=pd.read_csv(result_file, sep='\t')
    current[['TF_name','number_sites','extra_stuff']]=current['TF_profile'].str.rsplit('.', n=2, expand=True)
    current = current[['TF_name','HighFreqRange']]

    current  = current.set_index('TF_name').T.reset_index(drop=True)
    current['bam_name']=item.rsplit('_Accessibility1KSitesAdjusted.txt')[0]

    results = results.append(current, ignore_index=True)
features = results.set_index('bam_name').columns 
print('features', len(features))

features 504


In [4]:
print(len(results))
results.head()

540


TF_name,ADNP,AEBP2,AhR,Androgen,AP-2&#945;,AP-2&#947;,AP-4,ARID1A,ARID1B,ARID2,...,ZNF85,ZNF8,ZNF92,ZSCAN16,ZSCAN22,ZSCAN2,ZSCAN5A,ZSCAN5D,ZXDC,bam_name
0,0.246933,0.246460,0.154958,0.206862,0.231068,0.164809,0.264694,0.192881,0.178545,0.233550,...,0.234941,0.235168,0.240041,0.352748,0.204694,0.177236,0.206160,0.199835,0.151435,PGDX18259P_WGS.sorted_processed
1,0.197544,0.292149,0.156324,0.183372,0.197434,0.140286,0.401576,0.181430,0.199875,0.162787,...,0.215244,0.254871,0.233812,0.311026,0.254315,0.151788,0.155140,0.256196,0.175019,PGDX2750P_WGS_X1.sorted_processed
2,0.209211,0.225146,0.251445,0.213004,0.172528,0.188227,0.273997,0.235666,0.187254,0.182486,...,0.248464,0.265353,0.219906,0.362596,0.189254,0.179967,0.245128,0.219038,0.248635,PGDX5882P_WGS_processed_downsamp
3,0.259626,0.238307,0.164468,0.194450,0.123776,0.182936,0.421222,0.137561,0.165446,0.147863,...,0.175389,0.228228,0.201729,0.340457,0.137220,0.155708,0.163189,0.278206,0.180995,PGDX8828P_WGS.sorted_processed
4,0.319653,0.189597,0.160685,0.182166,0.206212,0.193224,0.576673,0.188484,0.199149,0.204391,...,0.190820,0.255983,0.258541,0.363226,0.223981,0.200423,0.264635,0.199601,0.153409,PGDX3512P5_WGS.sorted_processed


In [5]:
metadata = pd.read_csv(metadata_path, sep='\t')
metadata['bam_name'] = metadata['file_name'].str.rsplit('.bam',1,expand=True)[0]
metadata = metadata[~(metadata['Sample Type']=='Human adult elutriated lymphocytes')] #drop the non-cfDNA samples

In [6]:
merged = results.merge(metadata[['bam_name','new_sample_name','Stage']], on = 'bam_name')
merged['sample_type'] = merged['new_sample_name'].str.rsplit('_CG',1,expand=True)[0]
merged['status'] = np.where(merged['sample_type']=='Healthy',0,1)
merged = merged.rename(columns = {'new_sample_name':'sample'})

tumor_fractions = pd.read_csv(tfx_path, sep = '\t')
tumor_fractions = tumor_fractions.rename(columns = {'tfx':'tumor_fraction'})

merged = merged.merge(tumor_fractions[['sample','tumor_fraction']], on = 'sample')

#rename the feature columns
merged = merged.rename(columns = {m:'Ulz_'+m for m in merged[features].columns})


In [7]:
print(len(merged))
merged.head()

423


,Ulz_ADNP,Ulz_AEBP2,Ulz_AhR,Ulz_Androgen,Ulz_AP-2&#945;,Ulz_AP-2&#947;,Ulz_AP-4,Ulz_ARID1A,Ulz_ARID1B,Ulz_ARID2,...,Ulz_ZSCAN2,Ulz_ZSCAN5A,Ulz_ZSCAN5D,Ulz_ZXDC,bam_name,sample,Stage,sample_type,status,tumor_fraction
0,0.246933,0.246460,0.154958,0.206862,0.231068,0.164809,0.264694,0.192881,0.178545,0.233550,...,0.177236,0.206160,0.199835,0.151435,PGDX18259P_WGS.sorted_processed,Healthy_CGPLH640,NaN,Healthy,0,0.00000
1,0.197544,0.292149,0.156324,0.183372,0.197434,0.140286,0.401576,0.181430,0.199875,0.162787,...,0.151788,0.155140,0.256196,0.175019,PGDX2750P_WGS_X1.sorted_processed,Breast_Cancer_CGPLBR24,II,Breast_Cancer,1,0.02752
2,0.209211,0.225146,0.251445,0.213004,0.172528,0.188227,0.273997,0.235666,0.187254,0.182486,...,0.179967,0.245128,0.219038,0.248635,PGDX5882P_WGS_processed_downsamp,Colorectal_Cancer_CGCRC292,IV,Colorectal_Cancer,1,0.05273
3,0.259626,0.238307,0.164468,0.194450,0.123776,0.182936,0.421222,0.137561,0.165446,0.147863,...,0.155708,0.163189,0.278206,0.180995,PGDX8828P_WGS.sorted_processed,Colorectal_Cancer_CGCRC341,IV,Colorectal_Cancer,1,0.05058
4,0.219229,0.307641,0.150478,0.239262,0.213007,0.203533,0.464607,0.204062,0.130070,0.184918,...,0.191341,0.201426,0.238394,0.204629,PGDX18251P_WGS.sorted_processed,Healthy_CGPLH324,NaN,Healthy,0,0.00000


In [8]:
merged.to_csv(output_path,sep='\t',index=False)

In [13]:
#check that there aren't any nulls
merged[['Ulz_'+m for m in features]].isnull().any().any()

False

In [10]:
print(len(merged))
merged.shape

423


(423, 510)

In [11]:
merged[['Ulz_'+feature for feature in features]].shape

(423, 504)

In [12]:
merged['Stage'].unique()

array([nan, 'II', 'IV', 'I', 'III', '0', 'X'], dtype=object)